# Audio Sample Conversion Benchmark Analysis

In [3]:
import pandas as pd
import re
import os

### Parse Benchmark Data

The benchmark data is in text format. Let's parse it to extract the key metrics:


In [4]:
def parse_benchmark_data(file_path):
    """
    Parse the benchmark data from the given file path into a DataFrame.
    """
    benchmark_data = []

    with open(file_path, "r") as f:
        content = f.read()

    # Find all benchmark sections
    benchmark_sections = re.findall(
        r"Benchmarking (.*?)time:\s+\[(.*?)\]", content, re.DOTALL
    )

    for section, time_part in benchmark_sections:
        # Extract the benchmark name
        benchmark_name = section.strip().split("/")[0]

        # Parse the conversion information
        match = re.match(
            r"Samples conversion (\w+) to (\w+) - (\d+)s - (\d+)Hz - (\d+)ch",
            benchmark_name,
        )
        if match:
            from_type, to_type, duration, sample_rate, channels = match.groups()

            # Parse time measurements
            time_values = re.findall(r"(\d+\.\d+) (\w+)", time_part)
            if time_values:
                min_time, unit = time_values[0]
                median_time, _ = time_values[1]
                max_time, _ = time_values[2]

                # Convert to consistent units (nanoseconds)
                unit_multiplier = {
                    "ps": 0.001,  # picoseconds to nanoseconds
                    "ns": 1,  # nanoseconds
                    "µs": 1000,  # microseconds to nanoseconds
                    "ms": 1000000,  # milliseconds to nanoseconds
                }

                min_time_ns = float(min_time) * unit_multiplier.get(unit, 1)
                median_time_ns = float(median_time) * unit_multiplier.get(unit, 1)
                max_time_ns = float(max_time) * unit_multiplier.get(unit, 1)

                benchmark_data.append(
                    {
                        "from_type": from_type,
                        "to_type": to_type,
                        "duration_sec": int(duration),
                        "sample_rate_hz": int(sample_rate),
                        "channels": int(channels),
                        "min_time_ns": min_time_ns,
                        "median_time_ns": median_time_ns,
                        "max_time_ns": max_time_ns,
                        "time_unit": unit,
                    }
                )

    return pd.DataFrame(benchmark_data)

In [5]:
# Load the benchmark data
benchmark_df = parse_benchmark_data("conversion_benches.bench")

# Show the first few rows
print(f"Total number of benchmarks: {len(benchmark_df)}")
benchmark_df.head()

os.makedirs("./bench_results", exist_ok=True)

for group_name, group in benchmark_df.groupby(["from_type", "to_type"]):
    from_type, to_type = group_name
    fp = f"./bench_results/{from_type}_to_{to_type}.md"
    group.to_markdown(fp, tablefmt="github", index=False)

Total number of benchmarks: 180
